# Deep Learning Regression Model with Keras


## Objective: 

This project builds a regression model using the deep learning Keras library, and then experiments with increasing the number of training epochs and changing number of hidden layers and to see how changing these parameters impacts the performance of the model.

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement

2. Blast Furnace Slag</strong>

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

### 1. Downloading required libraries

In [4]:
# All Libraries required for this lab are listed below. The libraries pre-installed. 

#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [2]:
import pandas as pd
import numpy as np

### 2. Downloading the data and reading it into a `pandas` dataframe.


In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


Checking the number of data points 


In [5]:
concrete_data.shape

(1030, 9)

There are approximately 1000 samples to train the model on. Because of the few samples, there should be caution not to overfit the training data.


Checking the dataset for any missing values.


In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data has no missing values and appears to be very clean and is ready to be used to build our model.


### 3. Spliting data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, the predictors (features or x variables) will be all the other columns.


In [9]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Checking the predictors and the target dataframes.


In [10]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [11]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Normalizing the data by substracting the mean and dividing by the standard deviation.


In [13]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Saving the number of predictors to *n_cols* since we will need this number when building our network.


In [15]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>


<a id='item32'></a>


### 4. Importing Keras


note: Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first usually


In [16]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Importing the rest of the packages from the Keras library that we will need to build our regressoin model.


In [17]:
from keras.models import Sequential
from keras.layers import Dense

### A. Building a Neural Network


Defining a function that defines the regression model so that we can conveniently call it to create our model.The model is composed of: 

* 1 hidden layer of 10 nodes (i.e., neurons or perceptrons), and a ReLU activation function 
* Adam Optimizer and mean squared error loss function



In [18]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    # 1 hidden layer with 10 nodes and ReLU activation
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))  # Change the number of nodes to 10
    # Output layer (1 node)
    model.add(Dense(1))
    # compile model with Adam optimizer and mean squared error loss
    model.compile(optimizer='adam', loss='mean_squared_error')  # Use 'adam' optimizer and 'mean_squared_error' loss
    return model

#### Training and Testing the Network


Calling the above function now to create our model.


In [19]:
# build the model
model = regression_model()

Training and testing the model at the same time using the *fit* method. We will leave out 30% of the data for testing and we will train the model for 50 epochs.


In [23]:
# fit the model
#model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

from sklearn.model_selection import train_test_split

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=1)

# Fit the model to the training data for 50 epochs
model.fit(X_train, y_train, epochs=50)

# Evaluate the model on the testing data
loss = model.evaluate(X_test, y_test)

print("Mean Squared Error on Test Data:", loss)

Epoch 1/50
721/721 [==============================] - 0s 99us/step - loss: 18507578.3786
Epoch 2/50
721/721 [==============================] - 0s 81us/step - loss: 9360682.9743
Epoch 3/50
721/721 [==============================] - 0s 119us/step - loss: 4477258.9872
Epoch 4/50
721/721 [==============================] - 0s 106us/step - loss: 2072541.6325
Epoch 5/50
721/721 [==============================] - 0s 101us/step - loss: 962672.9264
Epoch 6/50
721/721 [==============================] - 0s 89us/step - loss: 487830.8703
Epoch 7/50
721/721 [==============================] - 0s 108us/step - loss: 307846.5430
Epoch 8/50
721/721 [==============================] - 0s 107us/step - loss: 241207.8563
Epoch 9/50
721/721 [==============================] - 0s 107us/step - loss: 217296.1614
Epoch 10/50
721/721 [==============================] - 0s 110us/step - loss: 206340.2728
Epoch 11/50
721/721 [==============================] - 0s 111us/step - loss: 199145.3784
Epoch 12/50
721/721 [=======

### B Training and Testing the Network with Normalised Data

Replacing x variables to normalised version: predictors_norm

In [24]:
# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=1)

# Fit the model to the training data for 50 epochs
model.fit(X_train, y_train, epochs=50)

# Evaluate the model on the testing data
loss = model.evaluate(X_test, y_test)

print("Mean Squared Error on Test Data:", loss)

Epoch 1/50
721/721 [==============================] - 0s 91us/step - loss: 219.6911
Epoch 2/50
721/721 [==============================] - 0s 89us/step - loss: 216.1510
Epoch 3/50
721/721 [==============================] - 0s 103us/step - loss: 212.6212
Epoch 4/50
721/721 [==============================] - 0s 87us/step - loss: 209.5319
Epoch 5/50
721/721 [==============================] - 0s 89us/step - loss: 206.6669
Epoch 6/50
721/721 [==============================] - 0s 105us/step - loss: 204.1028
Epoch 7/50
721/721 [==============================] - 0s 90us/step - loss: 201.7664
Epoch 8/50
721/721 [==============================] - 0s 80us/step - loss: 199.5429
Epoch 9/50
721/721 [==============================] - 0s 87us/step - loss: 197.5564
Epoch 10/50
721/721 [==============================] - 0s 90us/step - loss: 195.4475
Epoch 11/50
721/721 [==============================] - 0s 88us/step - loss: 193.5339
Epoch 12/50
721/721 [==============================] - 0s 98us/step - lo

Mean squared error on test data is now much lower

### C Training and Testing the Network with Normalised Data and 100 epochs 

In [25]:
# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=1)

# Fit the model to the training data for 50 epochs
model.fit(X_train, y_train, epochs=100)

# Evaluate the model on the testing data
loss = model.evaluate(X_test, y_test)

print("Mean Squared Error on Test Data:", loss)

Epoch 1/100
721/721 [==============================] - 0s 96us/step - loss: 134.2134
Epoch 2/100
721/721 [==============================] - 0s 142us/step - loss: 133.1263
Epoch 3/100
721/721 [==============================] - 0s 87us/step - loss: 132.0580
Epoch 4/100
721/721 [==============================] - 0s 103us/step - loss: 131.0096
Epoch 5/100
721/721 [==============================] - 0s 86us/step - loss: 130.0013
Epoch 6/100
721/721 [==============================] - 0s 86us/step - loss: 128.9798
Epoch 7/100
721/721 [==============================] - 0s 110us/step - loss: 127.9791
Epoch 8/100
721/721 [==============================] - 0s 90us/step - loss: 127.0206
Epoch 9/100
721/721 [==============================] - 0s 102us/step - loss: 125.9498
Epoch 10/100
721/721 [==============================] - 0s 83us/step - loss: 125.0035
Epoch 11/100
721/721 [==============================] - 0s 90us/step - loss: 123.9465
Epoch 12/100
721/721 [==============================] - 0s 

Mean Squared error has further reduced 

### D Training and Testing the Network with Normalised Data, 50 epochs, and new model consisting of 3 hidden layers, of 10 nodes each, and ReLU activation function

In [28]:
def regression_model():
    # create model
    model = Sequential()
    # First hidden layer with 10 nodes and ReLU activation
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    # Second hidden layer with 10 nodes and ReLU activation
    model.add(Dense(10, activation='relu'))
    # Third hidden layer with 10 nodes and ReLU activation
    model.add(Dense(10, activation='relu'))
    # Output layer (1 node)
    model.add(Dense(1))
    # compile model with Adam optimizer and mean squared error loss
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [29]:
# build the model
model = regression_model()

In [30]:
# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=1)

# Fit the model to the training data for 50 epochs
model.fit(X_train, y_train, epochs=50)

# Evaluate the model on the testing data
loss = model.evaluate(X_test, y_test)

print("Mean Squared Error on Test Data:", loss)

Epoch 1/50
721/721 [==============================] - 0s 510us/step - loss: 1491.8486
Epoch 2/50
721/721 [==============================] - 0s 170us/step - loss: 1452.6857
Epoch 3/50
721/721 [==============================] - 0s 165us/step - loss: 1393.8917
Epoch 4/50
721/721 [==============================] - 0s 189us/step - loss: 1301.0479
Epoch 5/50
721/721 [==============================] - 0s 170us/step - loss: 1156.5201
Epoch 6/50
721/721 [==============================] - 0s 163us/step - loss: 960.6498
Epoch 7/50
721/721 [==============================] - 0s 137us/step - loss: 725.3666
Epoch 8/50
721/721 [==============================] - 0s 156us/step - loss: 505.7230
Epoch 9/50
721/721 [==============================] - 0s 138us/step - loss: 352.3506
Epoch 10/50
721/721 [==============================] - 0s 118us/step - loss: 281.9698
Epoch 11/50
721/721 [==============================] - 0s 131us/step - loss: 250.2184
Epoch 12/50
721/721 [==============================] - 0s 

In [ ]:
The Mean Squared Error has increased meaning the original model wit